In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Normalize the pixel values between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert the labels to one-hot encoding
y_train = keras.utils.to_categorical(y_train, 100)
y_test = keras.utils.to_categorical(y_test, 100)


In [1]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the path to the directory
directory_path = r"E:\imbcifar\train"

# Initialize empty lists for images and labels
X_train = []
Y_train = []
class_label = 0

# Loop through subdirectories (classes)
for class_folder in tqdm(sorted(os.listdir(directory_path))):
    class_path = os.path.join(directory_path, class_folder)
    
    for image_file in os.listdir(class_path):
        if image_file.endswith('.png'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((32, 32))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_train.append(img_array)
            Y_train.append(class_label)
            
    class_label = class_label + 1

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.11it/s]

X_train shape: (29009, 32, 32, 3)
Y_train shape: (29009,)


In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tqdm import tqdm

# Load CIFAR-100 dataset
(train_data, test_data), ds_info = tfds.load('cifar100', split=['train', 'test'], 
                                             shuffle_files=True, as_supervised=True,
                                             with_info=True)

# Function to create class imbalance
def create_class_imbalance(dataset, imbalance_ratio=0.1):
    class_counts = [500] * 100  # CIFAR-100 has 500 images per class

    # Reduce the number of images for every alternate class
    imbalanced_class_counts = [int(count * imbalance_ratio) if i % 2 == 0 else count
                               for i, count in enumerate(class_counts)]

    imbalanced_dataset = []
    class_counter = {i: 0 for i in range(100)}  # Track class counts

    # Using tqdm to show progress
    for image, label in tqdm(dataset, desc="Processing Imbalance"):
        label = label.numpy()  # Convert label to numpy for comparison
        if class_counter[label] < imbalanced_class_counts[label]:
            imbalanced_dataset.append((image, label))
            class_counter[label] += 1

    return imbalanced_dataset

# Create imbalanced CIFAR-100 dataset
imbalanced_train_data = create_class_imbalance(train_data)

# Preprocessing function (normalize)
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]
    return image, label

# Apply normalization to the imbalanced dataset
x_train, y_train = [], []

for image, label in tqdm(imbalanced_train_data, desc="Normalizing Data"):
    image, label = preprocess(image, label)
    x_train.append(image.numpy())  # Convert image to numpy array
    y_train.append(label)  # Directly append the label since it's already an integer

X_train = np.array(x_train)
y_train = np.array(y_train)

# Ready for TensorFlow training
print(f'x_train shape: {X_train.shape}, y_train shape: {y_train.shape}')


Normalizing Data: 100%|████████████████████████████████████████████████████████| 27500/27500 [00:11<00:00, 2333.56it/s]


x_train shape: (27500, 32, 32, 3), y_train shape: (27500,)


In [3]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
import glob

target_size = (32, 32)  # Change the values as per your requirement
# Load the pre-trained ResNet50 model with modified input shape
model = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(target_size[0], target_size[1], 3))

In [1]:
import pandas as pd
# Define the file path
file_path = r"D:\feature.csv"
# Read the tab-separated CSV file into a DataFrame
df = pd.read_csv(file_path, delimiter='\t')
columns_to_drop = [df.columns[0], df.columns[-1]]
data = df.drop(columns_to_drop, axis=1)
# Display the head of the DataFrame
data.head(1)

,Feature 0,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,...,Feature 2038,Feature 2039,Feature 2040,Feature 2041,Feature 2042,Feature 2043,Feature 2044,Feature 2045,Feature 2046,Feature 2047
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.009121,0.0,0.0,0.0,0.0,3.285227,0.0,0.0


In [4]:
np.shape(X_train)

(4357, 32, 32, 3)

In [5]:
from PIL import Image
import numpy as np
ft = model.predict(np.array(X_train).astype("float32"))

In [6]:
from sklearn.cluster import MiniBatchKMeans

n_clusters = 400
batch_size = 100
max_iter = 100

kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=batch_size, max_iter=max_iter)
kmeans.fit(ft)
# Retrieve the cluster centers
ct = kmeans.cluster_centers_.tolist()

C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\sklearn\cluster\_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


In [7]:
from tqdm import tqdm

combo_list = []  # Initialize combo_list
tot_dist = []
# Iterate over ft
for i in tqdm(range(len(ct))):
    distances = []

    # Calculate distances for each row in data
    for index, row in data.iterrows():
        row_array = row.to_numpy()  # Convert row to numpy array
        distance = np.linalg.norm(ct[i] - row_array)  # Calculate Euclidean distance
        distances.append(distance)

    tot_dist.append(distances)
print(np.shape(tot_dist))
#new_shape = (1000, 30)
#re_dist = np.transpose(tot_dist, (1, 0))
re_dist = tot_dist
print(np.shape(re_dist))

100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [01:29<00:00,  4.49it/s]


(400, 1000)
(400, 1000)


In [8]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"D:\dist_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(re_dist, file)

print("Combo list saved successfully.")

Combo list saved successfully.


In [9]:
len(index_list)

NameError: name 'index_list' is not defined

In [2]:
import pickle

# Loading the list from the pickle file
file_path = r"D:\index_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    index_list = pickle.load(file)

print("Combo list loaded successfully.")
print(index_list)


Combo list loaded successfully.
[0, 1, 2, 3, 4, 5, 6, 10, 14, 18, 20, 21, 25, 27, 29, 31, 33, 34, 41, 44, 45, 49, 50, 55, 57, 58, 59, 65, 66, 70, 73, 78, 80, 81, 83, 85, 86, 92, 93, 96, 98, 103, 105, 106, 107, 108, 109, 115, 117, 123, 127, 128, 129, 131, 132, 133, 136, 137, 138, 139, 140, 141, 142, 143, 147, 148, 149, 152, 153, 155, 156, 157, 159, 165, 166, 179, 181, 183, 188, 190, 195, 197, 199, 205, 206, 210, 213, 214, 216, 217, 221, 224, 225, 229, 232, 239, 240, 241, 244, 247, 252, 256, 268, 272, 274, 276, 277, 278, 280, 291, 293, 299, 302, 306, 314, 315, 317, 318, 320, 321, 324, 325, 326, 330, 331, 332, 343, 344, 347, 350, 351, 352, 353, 361, 363, 368, 369, 375, 379, 384, 388, 392, 393, 397, 399, 400, 403, 404, 406, 408, 409, 411, 413, 414, 417, 424, 426, 431, 432, 436, 439, 441, 444, 447, 449, 450, 451, 453, 457, 458, 460, 465, 466, 470, 472, 475, 479, 482, 483, 484, 493, 495, 498, 500, 501, 510, 511, 512, 518, 519, 520, 525, 527, 528, 529, 531, 532, 533, 535, 536, 537, 539, 540, 

In [5]:
df.columns

Index(['Directory', 'Feature 0', 'Feature 1', 'Feature 2', 'Feature 3',
       'Feature 4', 'Feature 5', 'Feature 6', 'Feature 7', 'Feature 8',
       ...
       'Feature 2039', 'Feature 2040', 'Feature 2041', 'Feature 2042',
       'Feature 2043', 'Feature 2044', 'Feature 2045', 'Feature 2046',
       'Feature 2047', 'Unnamed: 2049'],
      dtype='object', length=2050)

In [3]:
selected_data = df.loc[index_list, ['Name']].to_numpy().flatten()

In [4]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"D:\combo_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(selected_data, file)

print("Combo list saved successfully.")


Combo list saved successfully.


In [5]:
import pickle

# Loading the list from the pickle file
file_path = r"D:\combo_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    loaded_combo_list = pickle.load(file)

print("Combo list loaded successfully.")
print(loaded_combo_list)


Combo list loaded successfully.
['D:/data/imagenet\\n01440764\\' 'D:/data/imagenet\\n01443537\\'
 'D:/data/imagenet\\n01484850\\' 'D:/data/imagenet\\n01491361\\'
 'D:/data/imagenet\\n01494475\\' 'D:/data/imagenet\\n01496331\\'
 'D:/data/imagenet\\n01498041\\' 'D:/data/imagenet\\n01530575\\'
 'D:/data/imagenet\\n01537544\\' 'D:/data/imagenet\\n01582220\\'
 'D:/data/imagenet\\n01601694\\' 'D:/data/imagenet\\n01608432\\'
 'D:/data/imagenet\\n01629819\\' 'D:/data/imagenet\\n01631663\\'
 'D:/data/imagenet\\n01632777\\' 'D:/data/imagenet\\n01644373\\'
 'D:/data/imagenet\\n01664065\\' 'D:/data/imagenet\\n01665541\\'
 'D:/data/imagenet\\n01685808\\' 'D:/data/imagenet\\n01689811\\'
 'D:/data/imagenet\\n01692333\\' 'D:/data/imagenet\\n01697457\\'
 'D:/data/imagenet\\n01698640\\' 'D:/data/imagenet\\n01729977\\'
 'D:/data/imagenet\\n01735189\\' 'D:/data/imagenet\\n01737021\\'
 'D:/data/imagenet\\n01739381\\' 'D:/data/imagenet\\n01751748\\'
 'D:/data/imagenet\\n01753488\\' 'D:/data/imagenet\\n01770

In [5]:
import os

file_path = r"F:\D-Video\Python"

# Check if the file exists
if os.path.exists(file_path):
    print(f"File exists: {file_path}")
else:
    print(f"File does not exist: {file_path}")

# Check if the file is readable
if os.access(file_path, os.R_OK):
    print(f"File is readable: {file_path}")
else:
    print(f"File is not readable: {file_path}")


File exists: F:\D-Video\Python
File is readable: F:\D-Video\Python
